# Binary Logistic Regression

In [1]:
# Binary Classification Demo: Logistic Regression (Mock dataset)
# - Pure Python + scikit-learn
# - Prints accuracy + confusion matrix + learned weights

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------
# 1) Mock (synthetic) dataset
# ----------------------------
# We generate 2 numeric features:
#   x1 = "study_hours"
#   x2 = "sleep_hours"
# Label rule (with noise):
#   more study + decent sleep -> higher chance to pass (y=1)
rng = np.random.default_rng(42)
N = 400

study_hours = rng.uniform(0, 10, size=N)   # 0..10
sleep_hours = rng.uniform(3, 9, size=N)    # 3..9

# Linear score + noise
noise = rng.normal(0, 1.2, size=N)
score = 1.1 * study_hours + 0.7 * sleep_hours - 8.0 + noise

# Convert score -> probability via sigmoid, then sample label
prob = 1.0 / (1.0 + np.exp(-score))
y = (rng.uniform(0, 1, size=N) < prob).astype(int)

# Feature matrix
X = np.column_stack([study_hours, sleep_hours])

print("Dataset shape:", X.shape, "Labels (0/1) counts:", np.bincount(y))

# ----------------------------
# 2) Train / Test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7, stratify=y
)

# ----------------------------
# 3) Train Logistic Regression
# ----------------------------
# solver='liblinear' is great for small binary problems
model = LogisticRegression()
model.fit(X_train, y_train)

# ----------------------------
# 4) Evaluate
# ----------------------------
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n=== Test Results ===")
print("Accuracy:", round(acc, 4))
print("Confusion matrix [[TN FP],[FN TP]]:\n", cm)
print("\nClassification report:\n", classification_report(y_test, y_pred, digits=4))

# ----------------------------
# 5) Show learned parameters
# ----------------------------
print("Intercept (b):", model.intercept_[0])
print("Weights (w1, w2):", model.coef_[0])
print("Feature order: [study_hours, sleep_hours]")

# ----------------------------
# 6) Predict a few new samples
# ----------------------------
new_X = np.array([
    [2.0, 4.0],   # low study, low sleep
    [8.0, 6.0],   # high study, ok sleep
    [5.0, 8.0],   # mid study, high sleep
], dtype=float)

pred_class = model.predict(new_X)
pred_prob = model.predict_proba(new_X)[:, 1]

print("\n=== New samples ===")
for i, (x, c, p) in enumerate(zip(new_X, pred_class, pred_prob), start=1):
    print(f"Sample {i}: study={x[0]:.1f}, sleep={x[1]:.1f} -> class={c}, P(y=1)={p:.3f}")


Dataset shape: (400, 2) Labels (0/1) counts: [142 258]

=== Test Results ===
Accuracy: 0.89
Confusion matrix [[TN FP],[FN TP]]:
 [[28  7]
 [ 4 61]]

Classification report:
               precision    recall  f1-score   support

           0     0.8750    0.8000    0.8358        35
           1     0.8971    0.9385    0.9173        65

    accuracy                         0.8900       100
   macro avg     0.8860    0.8692    0.8766       100
weighted avg     0.8893    0.8900    0.8888       100

Intercept (b): -5.873737079819337
Weights (w1, w2): [0.76603786 0.5501515 ]
Feature order: [study_hours, sleep_hours]

=== New samples ===
Sample 1: study=2.0, sleep=4.0 -> class=0, P(y=1)=0.105
Sample 2: study=8.0, sleep=6.0 -> class=1, P(y=1)=0.972
Sample 3: study=5.0, sleep=8.0 -> class=1, P(y=1)=0.914


# Multinomial Logistic Regression

In [2]:
# Logistic Regression on Iris (Binary + Multiclass demo)
# - Binary: setosa vs not-setosa
# - Multiclass: all 3 classes
# - Prints accuracy, confusion matrix, and learned weights

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------
# 1) Load Iris
# ----------------------------
iris = load_iris()
X = iris.data                       # (150, 4)
y_multi = iris.target               # 0=setosa, 1=versicolor, 2=virginica
feature_names = iris.feature_names

print("X shape:", X.shape)
print("Classes:", iris.target_names)

# =========================================================
# Multiclass Logistic Regression: 3-class classification
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y_multi, test_size=0.25, random_state=7, stratify=y_multi
)

# lbfgs supports multinomial well
multi_model = LogisticRegression(
    solver="lbfgs",
    multi_class="multinomial",
    max_iter=200,
    random_state=7
)
multi_model.fit(X_train, y_train)

y_pred = multi_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n====================")
print("MULTICLASS: setosa vs versicolor vs virginica")
print("====================")
print("Accuracy:", round(acc, 4))
print("Confusion matrix:\n", cm)
print("\nReport:\n", classification_report(y_test, y_pred, target_names=iris.target_names, digits=4))

print("Intercepts (b) per class:", multi_model.intercept_)
print("Weights (rows=classes, cols=features):")
for class_idx, class_name in enumerate(iris.target_names):
    print(f"\nClass: {class_name}")
    for name, w in zip(feature_names, multi_model.coef_[class_idx]):
        print(f"  {name:25s} {w:+.4f}")

# ----------------------------
# Predict a sample (optional)
# ----------------------------
sample = np.array([[5.9, 3.0, 5.1, 1.8]])  # one iris sample
pred_class = multi_model.predict(sample)[0]
pred_prob = multi_model.predict_proba(sample)[0]

print("\nSample:", sample[0])
print("Predicted class:", iris.target_names[pred_class])
print("Class probabilities:", dict(zip(iris.target_names, np.round(pred_prob, 4))))


X shape: (150, 4)
Classes: ['setosa' 'versicolor' 'virginica']

MULTICLASS: setosa vs versicolor vs virginica
Accuracy: 0.9737
Confusion matrix:
 [[13  0  0]
 [ 0 12  1]
 [ 0  0 12]]

Report:
               precision    recall  f1-score   support

      setosa     1.0000    1.0000    1.0000        13
  versicolor     1.0000    0.9231    0.9600        13
   virginica     0.9231    1.0000    0.9600        12

    accuracy                         0.9737        38
   macro avg     0.9744    0.9744    0.9733        38
weighted avg     0.9757    0.9737    0.9737        38

Intercepts (b) per class: [  9.15004074   1.91037397 -11.06041471]
Weights (rows=classes, cols=features):

Class: setosa
  sepal length (cm)         -0.4167
  sepal width (cm)          +0.8667
  petal length (cm)         -2.2962
  petal width (cm)          -0.9919

Class: versicolor
  sepal length (cm)         +0.5389
  sepal width (cm)          -0.3800
  petal length (cm)         -0.2111
  petal width (cm)          -0.740

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


--------------------------------------------
--------------------------------------------
--------------------------------------------

# SVM

In [3]:
# Multiclass (Multinomial-style) Classification with Linear SVM on Iris
# StandardScaler is applied EXPLICITLY (separate from model)
# NOTE: LinearSVC uses one-vs-rest (OvR) under the hood for multiclass.

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

# ----------------------------
# 1) Load Iris (3 classes)
# ----------------------------
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

# ----------------------------
# 2) Train / Test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7, stratify=y
)

# ----------------------------
# 3) Standard Scaling 
# ----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------------
# 4) Train Linear SVM (Multiclass)
# ----------------------------
svm = LinearSVC(C=1.0, random_state=7, max_iter=10000)  # multiclass handled internally
svm.fit(X_train_scaled, y_train)

# ----------------------------
# 5) Evaluate
# ----------------------------
y_pred = svm.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n====================")
print("LINEAR SVM (Multiclass): 3-class Iris")
print("====================")
print("Accuracy:", round(acc, 4))
print("Confusion matrix (rows=true, cols=pred):\n", cm)
print("\nReport:\n", classification_report(y_test, y_pred, target_names=class_names, digits=4))

# ----------------------------
# 6) Show learned parameters
# ----------------------------
# For multiclass LinearSVC:
#   coef_.shape = (n_classes, n_features)
#   intercept_.shape = (n_classes,)
print("coef_ shape:", svm.coef_.shape, "(classes x features)")
print("intercept_ shape:", svm.intercept_.shape, "(classes,)")

for k, cname in enumerate(class_names):
    print(f"\nClass '{cname}' hyperplane (in SCALED feature space):")
    print("  bias (b):", round(svm.intercept_[k], 4))
    for fname, w in zip(feature_names, svm.coef_[k]):
        print(f"  {fname:25s} {w:+.4f}")

# ----------------------------
# 7) Predict a new sample
# ----------------------------
sample = np.array([[5.9, 3.0, 5.1, 1.8]])
sample_scaled = scaler.transform(sample)

pred_class = svm.predict(sample_scaled)[0]
scores = svm.decision_function(sample_scaled)[0]  # one score per class

print("\nSample:", sample[0])
print("Predicted class:", class_names[pred_class])
print("Decision scores:", dict(zip(class_names, np.round(scores, 4))))



LINEAR SVM (Multiclass): 3-class Iris
Accuracy: 0.9474
Confusion matrix (rows=true, cols=pred):
 [[13  0  0]
 [ 0 11  2]
 [ 0  0 12]]

Report:
               precision    recall  f1-score   support

      setosa     1.0000    1.0000    1.0000        13
  versicolor     1.0000    0.8462    0.9167        13
   virginica     0.8571    1.0000    0.9231        12

    accuracy                         0.9474        38
   macro avg     0.9524    0.9487    0.9466        38
weighted avg     0.9549    0.9474    0.9472        38

coef_ shape: (3, 4) (classes x features)
intercept_ shape: (3,) (classes,)

Class 'setosa' hyperplane (in SCALED feature space):
  bias (b): -0.7997
  sepal length (cm)         -0.2036
  sepal width (cm)          +0.3788
  petal length (cm)         -0.6622
  petal width (cm)          -0.6772

Class 'versicolor' hyperplane (in SCALED feature space):
  bias (b): -0.376
  sepal length (cm)         -0.0607
  sepal width (cm)          -0.5040
  petal length (cm)         +0.5

----
----
----

# KNN

In [ ]:
# k-NN on Iris with CV to pick best k, then Train/Test + Predict
# - Step 1: CV on training set to choose best k
# - Step 2: Fit best model on training set
# - Step 3: Evaluate on test set + predict a new sample

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------
# 1) Load Iris
# ----------------------------
iris = load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# ----------------------------
# 2) Train/Test split (keep test untouched)
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7, stratify=y
)

# ----------------------------
# 3) Scale features (important for k-NN)
# ----------------------------
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# ----------------------------
# 4) Cross-validation to choose best k
# ----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

k_list = list(range(1, 31, 2))  # odd k: 1,3,5,...,29
mean_scores = []

for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_s, y_train, cv=cv, scoring="accuracy")
    mean_scores.append(scores.mean())

best_idx = int(np.argmax(mean_scores))
best_k = k_list[best_idx]
best_cv_acc = mean_scores[best_idx]

print("=== CV results (on TRAIN only) ===")
for k, m in zip(k_list, mean_scores):
    print(f"k={k:2d}  mean_CV_acc={m:.4f}")
print(f"\nBest k = {best_k}  (mean CV acc = {best_cv_acc:.4f})")

# ----------------------------
# 5) Train best k-NN on full training set
# ----------------------------
best_model = KNeighborsClassifier(n_neighbors=best_k)
best_model.fit(X_train_s, y_train)

# ----------------------------
# 6) Evaluate on test set
# ----------------------------
y_pred = best_model.predict(X_test_s)
test_acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n=== Test results ===")
print("Test accuracy:", round(test_acc, 4))
print("Confusion matrix (rows=true, cols=pred):\n", cm)
print("\nReport:\n", classification_report(y_test, y_pred, target_names=class_names, digits=4))

# ----------------------------
# 7) Predict a new sample
# ----------------------------
sample = np.array([[5.9, 3.0, 5.1, 1.8]])  # [sepal len, sepal wid, petal len, petal wid]
sample_s = scaler.transform(sample)

pred_class = best_model.predict(sample_s)[0]
pred_prob = best_model.predict_proba(sample_s)[0]

print("\n=== New sample prediction ===")
print("Sample:", sample[0])
print("Predicted class:", class_names[pred_class])
print("Probabilities:", dict(zip(class_names, np.round(pred_prob, 4))))


=== CV results (on TRAIN only) ===

Best k = 3  (mean CV acc = 0.9644)

=== Test results ===
Test accuracy: 0.9737
Confusion matrix (rows=true, cols=pred):
 [[13  0  0]
 [ 0 12  1]
 [ 0  0 12]]

Report:
               precision    recall  f1-score   support

      setosa     1.0000    1.0000    1.0000        13
  versicolor     1.0000    0.9231    0.9600        13
   virginica     0.9231    1.0000    0.9600        12

    accuracy                         0.9737        38
   macro avg     0.9744    0.9744    0.9733        38
weighted avg     0.9757    0.9737    0.9737        38


=== New sample prediction ===
Sample: [5.9 3.  5.1 1.8]
Predicted class: virginica
Probabilities: {np.str_('setosa'): np.float64(0.0), np.str_('versicolor'): np.float64(0.3333), np.str_('virginica'): np.float64(0.6667)}


---
---
---

# Decision Trees

In [5]:
# Decision Tree on Iris with CV to choose best max_depth
# - Step 1: CV on training set to pick best depth
# - Step 2: Fit best tree on full training set
# - Step 3: Evaluate on untouched test set + predict sample

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------
# 1) Load Iris
# ----------------------------
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

# ----------------------------
# 2) Train/Test split (keep test untouched)
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7, stratify=y
)

# ----------------------------
# 3) CV to choose best max_depth (on TRAIN only)
# ----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

depth_list = list(range(1, 11))  # 1..10
mean_scores = []

for d in depth_list:
    tree = DecisionTreeClassifier(criterion="gini", max_depth=d, random_state=7)
    scores = cross_val_score(tree, X_train, y_train, cv=cv, scoring="accuracy")
    mean_scores.append(scores.mean())

best_idx = int(np.argmax(mean_scores))
best_depth = depth_list[best_idx]
best_cv_acc = mean_scores[best_idx]

print("=== CV results (on TRAIN only) ===")
for d, m in zip(depth_list, mean_scores):
    print(f"max_depth={d:2d}  mean_CV_acc={m:.4f}")
print(f"\nBest max_depth = {best_depth}  (mean CV acc = {best_cv_acc:.4f})")

# ----------------------------
# 4) Train best Decision Tree on full training set
# ----------------------------
best_tree = DecisionTreeClassifier(criterion="gini", max_depth=best_depth, random_state=7)
best_tree.fit(X_train, y_train)

# ----------------------------
# 5) Evaluate on test set
# ----------------------------
y_pred = best_tree.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n=== Test results ===")
print("Test accuracy:", round(acc, 4))
print("Confusion matrix (rows=true, cols=pred):\n", cm)
print("\nReport:\n", classification_report(y_test, y_pred, target_names=class_names, digits=4))

# ----------------------------
# 6) Show best tree rules (text)
# ----------------------------
print("\n=== Best Tree rules (text) ===")
print(export_text(best_tree, feature_names=feature_names))

# ----------------------------
# 7) Predict a new sample
# ----------------------------
sample = np.array([[5.9, 3.0, 5.1, 1.8]])
pred_class = best_tree.predict(sample)[0]
pred_prob = best_tree.predict_proba(sample)[0]

print("\n=== New sample prediction ===")
print("Sample:", sample[0])
print("Predicted class:", class_names[pred_class])
print("Probabilities:", dict(zip(class_names, np.round(pred_prob, 4))))


=== CV results (on TRAIN only) ===
max_depth= 1  mean_CV_acc=0.6609
max_depth= 2  mean_CV_acc=0.9289
max_depth= 3  mean_CV_acc=0.9553
max_depth= 4  mean_CV_acc=0.9553
max_depth= 5  mean_CV_acc=0.9553
max_depth= 6  mean_CV_acc=0.9553
max_depth= 7  mean_CV_acc=0.9553
max_depth= 8  mean_CV_acc=0.9553
max_depth= 9  mean_CV_acc=0.9553
max_depth=10  mean_CV_acc=0.9553

Best max_depth = 3  (mean CV acc = 0.9553)

=== Test results ===
Test accuracy: 0.9737
Confusion matrix (rows=true, cols=pred):
 [[13  0  0]
 [ 0 12  1]
 [ 0  0 12]]

Report:
               precision    recall  f1-score   support

      setosa     1.0000    1.0000    1.0000        13
  versicolor     1.0000    0.9231    0.9600        13
   virginica     0.9231    1.0000    0.9600        12

    accuracy                         0.9737        38
   macro avg     0.9744    0.9744    0.9733        38
weighted avg     0.9757    0.9737    0.9737        38


=== Best Tree rules (text) ===
|--- petal length (cm) <= 2.45
|   |--- class

---
---
---

# Random Forests

In [6]:
# Random Forest on Iris with CV to choose best max_depth
# - Step 1: CV on training set to pick best depth
# - Step 2: Fit best RF on full training set
# - Step 3: Evaluate on untouched test set + predict sample
# - Also prints feature importances

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------
# 1) Load Iris
# ----------------------------
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

# ----------------------------
# 2) Train/Test split (keep test untouched)
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7, stratify=y
)

# ----------------------------
# 3) CV to choose best max_depth (on TRAIN only)
# ----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

depth_list = [None, 1, 2, 3, 4, 5, 6, 8, 10]
mean_scores = []

for d in depth_list:
    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=d,
        random_state=7,
        n_jobs=-1
    )
    scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring="accuracy")
    mean_scores.append(scores.mean())

best_idx = int(np.argmax(mean_scores))
best_depth = depth_list[best_idx]
best_cv_acc = mean_scores[best_idx]

print("=== CV results (on TRAIN only) ===")
for d, m in zip(depth_list, mean_scores):
    d_text = "None" if d is None else str(d)
    print(f"max_depth={d_text:>4s}  mean_CV_acc={m:.4f}")
print(f"\nBest max_depth = {('None' if best_depth is None else best_depth)}  (mean CV acc = {best_cv_acc:.4f})")

# ----------------------------
# 4) Train best Random Forest on full training set
# ----------------------------
best_rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=best_depth,
    random_state=7,
    n_jobs=-1
)
best_rf.fit(X_train, y_train)

# ----------------------------
# 5) Evaluate on test set
# ----------------------------
y_pred = best_rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("\n=== Test results ===")
print("Test accuracy:", round(acc, 4))
print("Confusion matrix (rows=true, cols=pred):\n", cm)
print("\nReport:\n", classification_report(y_test, y_pred, target_names=class_names, digits=4))

# ----------------------------
# 6) Feature importances
# ----------------------------
importances = best_rf.feature_importances_
order = np.argsort(importances)[::-1]

print("\n=== Feature importances (higher = more important) ===")
for idx in order:
    print(f"{feature_names[idx]:25s} {importances[idx]:.4f}")

# ----------------------------
# 7) Predict a new sample
# ----------------------------
sample = np.array([[5.9, 3.0, 5.1, 1.8]])
pred_class = best_rf.predict(sample)[0]
pred_prob = best_rf.predict_proba(sample)[0]

print("\n=== New sample prediction ===")
print("Sample:", sample[0])
print("Predicted class:", class_names[pred_class])
print("Probabilities:", dict(zip(class_names, np.round(pred_prob, 4))))


=== CV results (on TRAIN only) ===
max_depth=None  mean_CV_acc=0.9644
max_depth=   1  mean_CV_acc=0.9281
max_depth=   2  mean_CV_acc=0.9466
max_depth=   3  mean_CV_acc=0.9557
max_depth=   4  mean_CV_acc=0.9644
max_depth=   5  mean_CV_acc=0.9644
max_depth=   6  mean_CV_acc=0.9644
max_depth=   8  mean_CV_acc=0.9644
max_depth=  10  mean_CV_acc=0.9644

Best max_depth = None  (mean CV acc = 0.9644)

=== Test results ===
Test accuracy: 0.9737
Confusion matrix (rows=true, cols=pred):
 [[13  0  0]
 [ 0 12  1]
 [ 0  0 12]]

Report:
               precision    recall  f1-score   support

      setosa     1.0000    1.0000    1.0000        13
  versicolor     1.0000    0.9231    0.9600        13
   virginica     0.9231    1.0000    0.9600        12

    accuracy                         0.9737        38
   macro avg     0.9744    0.9744    0.9733        38
weighted avg     0.9757    0.9737    0.9737        38


=== Feature importances (higher = more important) ===
petal length (cm)         0.4542
p